In [37]:
import pandas as pd
import numpy as np
import matplotlib as pl
import sqlite3 as sql

In [38]:
october_data = pd.read_csv("discount_for_202010.csv", sep=';', comment='#', encoding='utf-8')
customers = pd.read_csv("md_customers.csv", sep=';', comment='#', encoding = 'cp1251')
sales = pd.read_csv("sales_data.csv", sep=";")

In [39]:
#fixing columns
october_data['PGI_DATE'] = pd.to_datetime(october_data['PGI_DATE'], format = '%Y%m%d')
sales['PGI_DATE'] = pd.to_datetime(sales['PGI_DATE'], format = '%Y%m%d')

# october_data['SHIP_TO'] = october_data['SHIP_TO'].apply(lambda x: int(x, 16))
# october_data['SKU'] = october_data['SKU'].apply(lambda x: int(x, 16))

# sales['SHIP_TO'] = sales['SHIP_TO'].apply(lambda x: int(x, 16))
# sales['SKU'] = sales['SKU'].apply(lambda x: int(x, 16))

if (type(sales['VOL'][0]) == str):
    sales['VOL'] = sales['VOL'].str.replace(',', '.').astype(float)
if (type(october_data['DISCOUNT'][0]) == str):
    october_data['DISCOUNT'] = october_data['DISCOUNT'].str.replace(',', '.').astype(float)

In [4]:
october_data

,PGI_DATE,SHIP_TO,SKU,DISCOUNT
0,2020-10-01,5CF75D24155048112638F87DB1D1226C,7DFA4250D2207EEB61883DDF5B67A1A7,22.00
1,2020-10-01,5CF75D24155048112638F87DB1D1226C,F2B3E6A656C805BF1CEB1FC5734E7622,20.00
2,2020-10-01,5CF75D24155048112638F87DB1D1226C,8266B80A417D8DF4267AA877838FBBFA,20.00
3,2020-10-01,5CF75D24155048112638F87DB1D1226C,37E9B839EEB8B2D3C484E37C548DCCAE,22.00
4,2020-10-01,5CF75D24155048112638F87DB1D1226C,32CD0B3CFED084C5C03F6AAFA0BB8D13,22.00
...,...,...,...,...
57640,2020-10-30,FC81E17774E555008D02A4D8D697AAF6,620CF34A42D15DEC58FF09697700AA63,15.61
57641,2020-10-30,FC81E17774E555008D02A4D8D697AAF6,65213079F85E2832DC71E5EEE3C09D6B,19.96
57642,2020-10-30,FC81E17774E555008D02A4D8D697AAF6,B0823E75BDA4AC493CF30862C2B977EF,31.00
57643,2020-10-30,FC81E17774E555008D02A4D8D697AAF6,F0281CEE08E07B19738CE6C21D01963F,21.00


In [5]:
sales

,PGI_DATE,SHIP_TO,SKU,VOL,DISCOUNT
0,2019-04-01,0025D90699DB5080150F0CF48DD1893E,02F67F20D3E7CFB0AB7F15DF72C65A9A,180.48720,NaN
1,2019-04-01,0025D90699DB5080150F0CF48DD1893E,090FFEA4BAF99F279A34AFFCD9421379,24.30360,NaN
2,2019-04-01,0025D90699DB5080150F0CF48DD1893E,1E67A23D1F1D65B8450CC03F9C6871C7,7.91280,NaN
3,2019-04-01,0025D90699DB5080150F0CF48DD1893E,1ECD1812B9FEABA0553E69ACCEECF54F,9.42000,NaN
4,2019-04-01,0025D90699DB5080150F0CF48DD1893E,1FB5AE8891CF6A8B1D32D3F1121B34F2,175.21200,NaN
...,...,...,...,...,...
946582,2020-04-30,FDE730A2CC12A0A83F25E13860A831F8,27D58363354F686E5B3AADAC379D4F34,-3.88104,NaN
946583,2020-04-30,FF4164A314FE16FA6752954835E44061,8D6BF8B63FD1C298BB2F319D09833A77,197.82000,NaN
946584,2020-04-30,FF4164A314FE16FA6752954835E44061,ACA7B2EA935B16E77ED105BD91C9A1DC,84.78000,NaN
946585,2020-04-30,FF4164A314FE16FA6752954835E44061,EB0B331C773E9CF898D652ACBF59B12B,83.08440,NaN


In [6]:
customers

,SHIP_TO,SALES_DISTRICT,SALES_OFFICE,SALES_GROUP,CUST_GRP,PROXI_CATEG2,PROXI_CATEG3,TERR_LVL2,TERR_LVL3
0,6B7CEEFAAE1027BDAD99F2C96119FE87,Дивизион 7,Город 16,Филиал 173,Формат ТТ 56,Торговая сеть 67,E-COM,Регион 5,Область 10
1,81752EA1CF02C19CC7B55561FFE0C760,Дивизион 7,Город 16,Филиал 173,Формат ТТ 56,Торговая сеть 67,E-COM,Регион 5,Область 10
2,706155A5C878094F242F042F11C16B1A,Дивизион 7,Город 16,Филиал 173,Формат ТТ 56,Торговая сеть 67,E-COM,Регион 5,Область 10
3,15D49593D9E57DA1491C91AEAD310E09,Дивизион 5,Город 24,Филиал 15,Формат ТТ 56,Торговая сеть 67,E-COM,Регион 4,Область 13
4,BA2A9FC607C7036B55C7055AB0D727B6,Дивизион 2,Город 14,Филиал 136,Формат ТТ 4,Торговая сеть 125,INDIRECT,Регион 7,Область 4
...,...,...,...,...,...,...,...,...,...
1268,D1254EE75E94D92A2B456AA8F29BA9DE,Дивизион 2,Город 17,Филиал 84,Формат ТТ 3,Торговая сеть 726,INDIRECT,Регион 7,Область 9
1269,B98406DC1322BE399E982D38E28AE2D1,Дивизион 2,Город 17,Филиал 84,Формат ТТ 3,Торговая сеть 726,INDIRECT,Регион 7,Область 9
1270,8BDD4F8D14144D7218B3CCC9C44616DC,Дивизион 5,Город 24,Филиал 103,Формат ТТ 3,Торговая сеть 728,INDIRECT,Регион 4,Область 13
1271,2579A834889B701006A94BA046ADEABF,Дивизион 5,Город 24,Филиал 103,Формат ТТ 3,Торговая сеть 728,INDIRECT,Регион 4,Область 13


### Скрипты для подсчета вспомогательной базы (Rita's script)

In [7]:
db = sql.connect('UB_WA.db')
with db:
    #creating MD_CUSTOMERS and VW_JDLV_SALES
    current = db.cursor()
    current.execute(
        "CREATE TABLE IF NOT EXISTS VW_JDLV_SALES("
        " PGI_DATE DATE, "
        "SHIP_TO VARCHAR(32), "
        "SKU VARCHAR(32), "
        "VOL REAL, "
        "DISCOUNT NUMBER); ")
    current.execute(
        "CREATE TABLE IF NOT EXISTS MD_CUSTOMERS ("
        "SHIP_TO VARCHAR(32), "
        "SALES_DISTRICT VARCHAR(32), "
        "SALES_OFFICE VARCHAR(32), "
        "SALES_GROUP VARCHAR(32),"
        "RETAIL_CHAIN VARCHAR(32),"
        "CUST_GRP VARCHAR(32),"
        "PROXI_CATEG2 VARCHAR(32) DEFAULT '##',"
        "PROXI_CATEG3 VARCHAR(32) DEFAULT '##',"
        "TERR_LVL2 VARCHAR(32) DEFAULT '##',"
        "TERR_LVL3 VARCHAR(32) DEFAULT '##'); ")
    
    #reading and moving MD_CUSTOMERS and VW_JDLV_SALES
    customers.to_sql('MD_CUSTOMERS', db, if_exists='replace', index=False)
    sales.to_sql('VW_JDLV_SALES', db, if_exists='replace', index=False)
    
    
    #help table. Count active dots with group by TERR_LVL2 and TERR_LVL3 
    current.execute("DROP TABLE IF EXISTS all_sales")
    current.execute(
        "CREATE TABLE IF NOT EXISTS all_sales AS "
        "SELECT "
        "SALES_DISTRICT, "
        "SALES_OFFICE, "
        "SALES_GROUP, "
        "MD_CUSTOMERS.TERR_LVL2, "
        "MD_CUSTOMERS.TERR_LVL3, "
        "count(MD_CUSTOMERS.SHIP_TO) AS ALVAL, "
        "sum(VW_JDLV_SALES.VOL) AS ALVOL "
        "FROM MD_CUSTOMERS "
        "JOIN VW_JDLV_SALES ON MD_CUSTOMERS.SHIP_TO = VW_JDLV_SALES.SHIP_TO "
        "WHERE VW_JDLV_SALES.VOL > 0 "
        "GROUP BY "
        "SALES_DISTRICT, "
        "SALES_OFFICE, "
        "SALES_GROUP, "
        "MD_CUSTOMERS.TERR_LVL2, "
        "MD_CUSTOMERS.TERR_LVL3;"
    )
    
    
    #help table. doing same as prev. but for sku_sales
    current.execute("DROP TABLE IF EXISTS sku_sales")
    current.execute(
        "CREATE TABLE IF NOT EXISTS sku_sales AS "
        "SELECT "
        "SALES_DISTRICT, "
        "SALES_OFFICE, "
        "SALES_GROUP, "
        "MD_CUSTOMERS.TERR_LVL2, "
        "MD_CUSTOMERS.TERR_LVL3, "
        "VW_JDLV_SALES.SKU, "
        "count(MD_CUSTOMERS.SHIP_TO) AS SKUVAL, "
        "sum(VW_JDLV_SALES.VOL) AS SKUVOL "
        "FROM MD_CUSTOMERS "
        "JOIN VW_JDLV_SALES ON MD_CUSTOMERS.SHIP_TO = VW_JDLV_SALES.SHIP_TO "
        "WHERE VW_JDLV_SALES.VOL > 0 "
        "GROUP BY "
        "SALES_DISTRICT, "
        "SALES_OFFICE, "
        "SALES_GROUP, "
        "MD_CUSTOMERS.TERR_LVL2, "
        "MD_CUSTOMERS.TERR_LVL3, "
        "VW_JDLV_SALES.SKU;"
    )

In [70]:
content = sql.connect("UB_WA.db")
VW_JDLV_SALES = pd.read_sql("SELECT * from VW_JDLV_SALES", content)
MD_CUSTOMERS = pd.read_sql("SELECT * from MD_CUSTOMERS", content)

### Полученные таблицы VW_JDLV_SALES и MD_CUSTOMERS

In [9]:
VW_JDLV_SALES

,PGI_DATE,SHIP_TO,SKU,VOL,DISCOUNT
0,2019-04-01 00:00:00,0025D90699DB5080150F0CF48DD1893E,02F67F20D3E7CFB0AB7F15DF72C65A9A,180.48720,None
1,2019-04-01 00:00:00,0025D90699DB5080150F0CF48DD1893E,090FFEA4BAF99F279A34AFFCD9421379,24.30360,None
2,2019-04-01 00:00:00,0025D90699DB5080150F0CF48DD1893E,1E67A23D1F1D65B8450CC03F9C6871C7,7.91280,None
3,2019-04-01 00:00:00,0025D90699DB5080150F0CF48DD1893E,1ECD1812B9FEABA0553E69ACCEECF54F,9.42000,None
4,2019-04-01 00:00:00,0025D90699DB5080150F0CF48DD1893E,1FB5AE8891CF6A8B1D32D3F1121B34F2,175.21200,None
...,...,...,...,...,...
946582,2020-04-30 00:00:00,FDE730A2CC12A0A83F25E13860A831F8,27D58363354F686E5B3AADAC379D4F34,-3.88104,None
946583,2020-04-30 00:00:00,FF4164A314FE16FA6752954835E44061,8D6BF8B63FD1C298BB2F319D09833A77,197.82000,None
946584,2020-04-30 00:00:00,FF4164A314FE16FA6752954835E44061,ACA7B2EA935B16E77ED105BD91C9A1DC,84.78000,None
946585,2020-04-30 00:00:00,FF4164A314FE16FA6752954835E44061,EB0B331C773E9CF898D652ACBF59B12B,83.08440,None


In [72]:
MD_CUSTOMERS

,SHIP_TO,SALES_DISTRICT,SALES_OFFICE,SALES_GROUP,CUST_GRP,PROXI_CATEG2,PROXI_CATEG3,TERR_LVL2,TERR_LVL3
0,6B7CEEFAAE1027BDAD99F2C96119FE87,Дивизион 7,Город 16,Филиал 173,Формат ТТ 56,Торговая сеть 67,E-COM,Регион 5,Область 10
1,81752EA1CF02C19CC7B55561FFE0C760,Дивизион 7,Город 16,Филиал 173,Формат ТТ 56,Торговая сеть 67,E-COM,Регион 5,Область 10
2,706155A5C878094F242F042F11C16B1A,Дивизион 7,Город 16,Филиал 173,Формат ТТ 56,Торговая сеть 67,E-COM,Регион 5,Область 10
3,15D49593D9E57DA1491C91AEAD310E09,Дивизион 5,Город 24,Филиал 15,Формат ТТ 56,Торговая сеть 67,E-COM,Регион 4,Область 13
4,BA2A9FC607C7036B55C7055AB0D727B6,Дивизион 2,Город 14,Филиал 136,Формат ТТ 4,Торговая сеть 125,INDIRECT,Регион 7,Область 4
...,...,...,...,...,...,...,...,...,...
1268,D1254EE75E94D92A2B456AA8F29BA9DE,Дивизион 2,Город 17,Филиал 84,Формат ТТ 3,Торговая сеть 726,INDIRECT,Регион 7,Область 9
1269,B98406DC1322BE399E982D38E28AE2D1,Дивизион 2,Город 17,Филиал 84,Формат ТТ 3,Торговая сеть 726,INDIRECT,Регион 7,Область 9
1270,8BDD4F8D14144D7218B3CCC9C44616DC,Дивизион 5,Город 24,Филиал 103,Формат ТТ 3,Торговая сеть 728,INDIRECT,Регион 4,Область 13
1271,2579A834889B701006A94BA046ADEABF,Дивизион 5,Город 24,Филиал 103,Формат ТТ 3,Торговая сеть 728,INDIRECT,Регион 4,Область 13


### Витирина
 Витрина представлена как таблицы all_sales и sku_sales <br>
 sku_sales - таблица содержащая количество активных точек и объемы продаж по всем продуктам.<br>
 all_sales - таблица содержащая количество активных точек и объем продаж по всем территориальным единицам.

In [73]:
sku_sales = pd.read_sql("SELECT * from sku_sales", content)
all_sales = pd.read_sql("SELECT * from all_sales", content)

In [69]:
sku_sales

,SALES_DISTRICT,SALES_OFFICE,SALES_GROUP,TERR_LVL2,TERR_LVL3,SKU,SKUVAL,SKUVOL
0,Дивизион 2,Город 11,Филиал 122,Регион 7,Область 3,019675DC4388481A22E3B26E52AB0049,11,48.98400
1,Дивизион 2,Город 11,Филиал 122,Регион 7,Область 3,0219CC78D456019EDB2FF06DDFD4FB6E,9,71.84320
2,Дивизион 2,Город 11,Филиал 122,Регион 7,Область 3,021D9026DAC687868287E514C3BFD2BD,3,17.89800
3,Дивизион 2,Город 11,Филиал 122,Регион 7,Область 3,02B90701612E92D91D4F7F367CD52A6A,50,6916.01328
4,Дивизион 2,Город 11,Филиал 122,Регион 7,Область 3,02F67F20D3E7CFB0AB7F15DF72C65A9A,112,18897.00984
...,...,...,...,...,...,...,...,...
31926,Дивизион 8,Город 7,Филиал 79,Регион 2,Область 21,FEE75B05309B665FB5A6EC3B56DDA5E1,19,84.78000
31927,Дивизион 8,Город 7,Филиал 79,Регион 2,Область 21,FEF046C8338382959ADB6AAA90B51D2D,7,45.21600
31928,Дивизион 8,Город 7,Филиал 79,Регион 2,Область 21,FEF6877671F3E7A8F4428114C5FEF1D8,7,17.58400
31929,Дивизион 8,Город 7,Филиал 79,Регион 2,Область 21,FF1D218CB29C3B93378E14DD2B586987,5,19.59360


In [75]:
all_sales

,SALES_DISTRICT,SALES_OFFICE,SALES_GROUP,TERR_LVL2,TERR_LVL3,ALVAL,ALVOL
0,Дивизион 2,Город 11,Филиал 122,Регион 7,Область 3,9222,5.839074e+05
1,Дивизион 2,Город 14,Филиал 108,Регион 7,Область 4,1991,4.035155e+05
2,Дивизион 2,Город 14,Филиал 136,Регион 7,Область 4,849,1.580725e+04
3,Дивизион 2,Город 14,Филиал 137,Регион 7,Область 4,4689,1.468042e+06
4,Дивизион 2,Город 14,Филиал 138,Регион 7,Область 4,8900,1.427839e+06
...,...,...,...,...,...,...,...
85,Дивизион 8,Город 13,Филиал 34,Регион 2,Область 21,29456,1.038668e+06
86,Дивизион 8,Город 7,Филиал 112,Регион 2,Область 21,4997,2.131334e+05
87,Дивизион 8,Город 7,Филиал 113,Регион 2,Область 21,2603,6.757217e+04
88,Дивизион 8,Город 7,Филиал 69,Регион 2,Область 21,3965,3.368919e+04


### Функция для получения скрипта, который выдает разрез

In [12]:
def generate_code(lvl, table_name):
    script = "CREATE TABLE " + table_name + " AS " \
        "SELECT " \
        "sku_sales." + lvl + ", " \
        "sku_sales.SKU, " \
        "(sum(sku_sales.SKUVOL)* 1.0 / sum(all_sales.ALVOL)) AS WD, " \
        "(sum(sku_sales.SKUVAL) * 1.0 / sum(all_sales.ALVAL))  AS ND " \
        "FROM sku_sales " \
        "JOIN all_sales ON sku_sales." + lvl + " = all_sales." + lvl + " " \
        "GROUP BY " \
        "sku_sales." + lvl + ", " \
        "sku_sales.SKU;"
    return script

### Присвойте LVL имя столбца, по которому производится срез.
#### Опционально: Присвойте SKU код продукта.

In [79]:
# example with required table TD_RES_ND_WD and TERR_LVL3

NAME = "TD_RES_ND_WD"
LVL = "TERR_LVL3"
SKU = "019675DC4388481A22E3B26E52AB0049"

In [80]:
current.execute("DROP TABLE IF EXISTS " + NAME)
current.execute(generate_code(LVL, NAME))
current.execute("""
    ALTER TABLE TD_RES_ND_WD ADD 
    TEAM VARCHAR(32) DEFAULT 'team41' NOT NULL;
    """)

new_SKU = ""
if SKU != "":
    new_SKU = " WHERE SKU= \"" + SKU + "\";"

TD_RES_ND_WD = pd.read_sql("SELECT * from TD_RES_ND_WD" + new_SKU, content)

## Полученный срез

In [81]:
TD_RES_ND_WD

,TERR_LVL2,SKU,WD,ND,TEAM
0,Регион 2,019675DC4388481A22E3B26E52AB0049,0.000445,0.001097,team41
1,Регион 3,019675DC4388481A22E3B26E52AB0049,0.000153,0.000743,team41
2,Регион 4,019675DC4388481A22E3B26E52AB0049,0.000492,0.000850,team41
3,Регион 5,019675DC4388481A22E3B26E52AB0049,0.000226,0.001036,team41
4,Регион 7,019675DC4388481A22E3B26E52AB0049,0.000095,0.000818,team41


# Прогноз спроса


подготовка данных к обработке

In [47]:
sales

,PGI_DATE,SHIP_TO,SKU,VOL,DISCOUNT,year,month,day
0,2019-04-01,0025D90699DB5080150F0CF48DD1893E,02F67F20D3E7CFB0AB7F15DF72C65A9A,180.48720,NaN,2019,4,1
1,2019-04-01,0025D90699DB5080150F0CF48DD1893E,090FFEA4BAF99F279A34AFFCD9421379,24.30360,NaN,2019,4,1
2,2019-04-01,0025D90699DB5080150F0CF48DD1893E,1E67A23D1F1D65B8450CC03F9C6871C7,7.91280,NaN,2019,4,1
3,2019-04-01,0025D90699DB5080150F0CF48DD1893E,1ECD1812B9FEABA0553E69ACCEECF54F,9.42000,NaN,2019,4,1
4,2019-04-01,0025D90699DB5080150F0CF48DD1893E,1FB5AE8891CF6A8B1D32D3F1121B34F2,175.21200,NaN,2019,4,1
...,...,...,...,...,...,...,...,...
946582,2020-04-30,FDE730A2CC12A0A83F25E13860A831F8,27D58363354F686E5B3AADAC379D4F34,-3.88104,NaN,2020,4,30
946583,2020-04-30,FF4164A314FE16FA6752954835E44061,8D6BF8B63FD1C298BB2F319D09833A77,197.82000,NaN,2020,4,30
946584,2020-04-30,FF4164A314FE16FA6752954835E44061,ACA7B2EA935B16E77ED105BD91C9A1DC,84.78000,NaN,2020,4,30
946585,2020-04-30,FF4164A314FE16FA6752954835E44061,EB0B331C773E9CF898D652ACBF59B12B,83.08440,NaN,2020,4,30


In [51]:
data2 = sales
data2['year'] = pd.DatetimeIndex(data2['PGI_DATE']).year
data2['month'] = pd.DatetimeIndex(data2['PGI_DATE']).month
data2['day'] = pd.DatetimeIndex(data2['PGI_DATE']).day

data3 = data2.groupby(['SHIP_TO', 'SKU', 'year', 'month'], as_index = False).sum()

data4 = data3[['SKU', 'year', 'month', 'VOL']]
data4 = data4.groupby(['SKU', 'year', 'month'], as_index = False).sum()

Выделение рынка e-com

In [48]:
customers_ecom = customers[customers['PROXI_CATEG3'] == "E-COM"]
e_com_product = data2[['SHIP_TO', 'SKU','year', 'month', 'day', 'VOL']]

e_com_product = e_com_product[e_com_product['SHIP_TO'].isin(customers_ecom['SHIP_TO'])]
e_com_product = e_com_product[['SHIP_TO', 'SKU', 'year', 'month', 'VOL']]
e_com_product = e_com_product.groupby(['SHIP_TO', 'SKU', 'year', 'month'], as_index = False).sum()

data4 = e_com_product[['SKU', 'year', 'month', 'VOL']]
data4 = data4.groupby(['SKU', 'year', 'month'], as_index = False).sum()

Находим сезонное отношение объема продаж для каждого товара.

In [57]:
year = 2019

vol_4_19  = data4[(data4['month'] == 4) & (data4['year'] == year)]
vol_10_19 = data4[(data4['month'] == 10) & (data4['year'] == year)]

otn_10_4 = pd.merge(vol_10_19, vol_4_19, on = ['SKU'])

otn_10_4 = otn_10_4[['SKU', 'month_x', 'VOL_x', 'month_y', 'VOL_y']]
otn_10_4['otnoshenie'] = otn_10_4['VOL_x'] / otn_10_4['VOL_y']

otn_10_4 = otn_10_4[['SKU', 'otnoshenie']]

Расчитываем среднее значение сезонности, чтобы поставить на место неизвестных значениий

In [58]:
year = 2020
vol_4_20 = data4[(data4['month'] == 4) & (data4['year'] == year)]
vol_4_20 = vol_4_20.merge(otn_10_4, how='outer', on=['SKU'])
awesome_number = vol_4_20.otnoshenie.mean()
awesome_number

1.2734896954119426

Предсказываем значение товаров на октябрь 2020 исходя из данных на апрель 2020 и сезонность каждого товара

In [60]:
e_com_product_ext = data2[['SHIP_TO', 'SKU','year', 'month', 'day', 'VOL']]

e_com_product_ext = e_com_product_ext[e_com_product_ext['SHIP_TO'].isin(customers_ecom['SHIP_TO'])]

e_com_product_ext = e_com_product_ext[e_com_product_ext['year'] == 2020]

e_com_product_ext = e_com_product_ext.merge(otn_10_4, how='outer', on=['SKU'])
e_com_product_ext.otnoshenie = e_com_product_ext.otnoshenie.fillna(awesome_number)
e_com_product_ext = e_com_product_ext[e_com_product_ext['year'] == 2020 ]
e_com_product_ext['FORECAST_VOL'] = e_com_product_ext['VOL'] * e_com_product_ext['otnoshenie']
e_com_product_ext

,SHIP_TO,SKU,year,month,day,VOL,otnoshenie,FORECAST_VOL
0,0498DAC30D1212289C900D8615E372DB,096D75470D0AD4CEED09052B4FFA0857,2020.0,4.0,1.0,37.6800,0.869426,32.759957
1,1833767ED979A3D64EBDC8C1FC5F225F,096D75470D0AD4CEED09052B4FFA0857,2020.0,4.0,1.0,37.6800,0.869426,32.759957
2,B0210BA9C6E650542D00123E1454CFF8,096D75470D0AD4CEED09052B4FFA0857,2020.0,4.0,1.0,37.6800,0.869426,32.759957
3,CBCB6AF86761321603178D9FD67B8BA2,096D75470D0AD4CEED09052B4FFA0857,2020.0,4.0,1.0,37.6800,0.869426,32.759957
4,5C47083012B4C2FBFA60CFB38DD4D76A,096D75470D0AD4CEED09052B4FFA0857,2020.0,4.0,2.0,37.6800,0.869426,32.759957
...,...,...,...,...,...,...,...,...
85102,4B959945011A7FB4798E2FA6225CEC0B,3359AF98C1402664FFCE551CBAF92573,2020.0,4.0,28.0,7.3476,1.273490,9.357093
85103,4B959945011A7FB4798E2FA6225CEC0B,3359AF98C1402664FFCE551CBAF92573,2020.0,4.0,29.0,14.6952,1.273490,18.714186
85104,9CD6112F15BC9B864C1D6D1677842091,2FA6382AC32DB77831E7AC6159820DA3,2020.0,4.0,28.0,25.4340,3.301438,83.968773
85105,949C978772603679E20A8580474C84B7,53ACA0B54FC5B90D7DEA3FA1E1866384,2020.0,4.0,30.0,79.8345,1.273490,101.668413
